# SQL - Анализ базы данных сервиса чтения книг

Наша компания купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных. В базе данных есть информация о книгах, авторах, издательствах, о пользовательских оценках книг и обзорах на книги.

## Этапы вополнения проекта:

1. Подключение к БД

2. Чтение таблиц из БД

3. Анализ данных

    3.1. Количество книг, выпущенных после 1 января 2000 года

    3.2. Количеcтво обзоров и средняя оценка книги

    3.3. Издатели с книгами, в которых более 50 страниц

    3.4. Авторы книг с самой высокой оценкой (оценок 50+)

    3.5. Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок

4. Описание выводов

## Подключение к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
#query = '''SELECT * FROM books LIMIT 5'''

#con=engine.connect()

#pd.io.sql.read_sql(sql=text(query), con = con)

## Чтение таблиц из БД

In [2]:
books = ''' SELECT *
            FROM books
            LIMIT 5
        '''
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
authors = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
publishers = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
pd.io.sql.read_sql(publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [5]:
ratings = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
reviews = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Анализ данных

## Количество книг, выпущенных после 1 января 2000 года

In [7]:
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
books_after_2000 = ''' SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(books_after_2000, con = engine) 

,count
0,819


После 1 января 2020 года было выпущено 819 книг.

## Количество обзоров и средняя оценка книги

In [9]:
rating_mean = '''select title,
       count(distinct review_id) review_cnt,
       round(avg(rating), 2) rating_avg
from books b 

left join ratings r on b.book_id = r.book_id
left join reviews rv on b.book_id = rv.book_id
     
group by b.book_id
order by rating_avg desc, 
         review_cnt desc
'''
pd.io.sql.read_sql(rating_mean, con = engine)

,title,review_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Для выполнения этого задания я использовала таблицы ratings и reviews. Средний рейтинг от 1,5 до 5,0. Книг без обзоров нет. 

## Издатели с книгами, в которых более 50 страниц

In [10]:
books_50 = ''' SELECT publisher, count(book_id) AS books_count
             FROM (SELECT books.*, publishers.publisher
             FROM books, publishers
             WHERE books.publisher_id = publishers.publisher_id) AS tbl
             WHERE num_pages > 50
             GROUP BY publisher
             ORDER BY books_count DESC 
             LIMIT 1
'''

pd.io.sql.read_sql(books_50, con = engine) 

,publisher,books_count
0,Penguin Books,42


Для выполнения этого задания я использовала таблицы books и publisher. Издательство Penguin Books выпустило 42 книги с количеством страниц больше 50.

## Авторы книг с самой высокой оценкой (оценок 50+)

In [11]:
pop_author = '''SELECT authors.author, avg(book_avg_ratings.book_rating_avg) as author_rating_avg
                    FROM books
                    INNER JOIN (SELECT book_id, avg(rating) as book_rating_avg
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(*) >= 50) AS book_avg_ratings
                    ON book_avg_ratings.book_id = books.book_id
                    INNER JOIN authors ON authors.author_id = books.author_id
                    GROUP BY authors.author
                    ORDER BY author_rating_avg desc
                    
'''

pd.io.sql.read_sql(pop_author, con = engine)

,author,author_rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


В топ-3 самых популярных авторов вошли Джоан Роулинг (автор книг о Гарри Поттере), Маркус Зусак (автор книги "Книжный двор") и Джон Рональд Руэл Толкин (автор книг "Хоббиты" и "Властелин колец"). Самая высокая оценка у Джоан Роулинг.

## Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок

In [12]:
users = '''SELECT AVG(tbl.count)
           FROM (SELECT total.username, COUNT(total.username)
                 FROM (SELECT username, COUNT(rating_id) AS count
                       FROM ratings
                       GROUP BY username) as total
                 JOIN reviews ON reviews.username = total.username
                 WHERE total.count > 50
                 GROUP BY total.username) as tbl
            
'''

pd.io.sql.read_sql(users, con = engine)

,avg
0,24.333333


В среднем от самых активных польщователей мы получаем по 24 обзора.

In [13]:
less_one = ''' SELECT extract("year" from publication_date::date), count(distinct(publisher_id)) as publish_cnt, count(distinct(book_id)) as book_cnt,
            sum(num_pages) as num_page_cnt
            FROM books
            group by extract("year" from publication_date::date)
            having count(distinct(book_id)) > 30
            order by extract("year" from publication_date::date)
            
            
        '''
pd.io.sql.read_sql(less_one, con = engine) 

,date_part,publish_cnt,book_cnt,num_page_cnt
0,1999.0,26,41,15763
1,2000.0,35,38,13328
2,2001.0,41,60,21758
3,2002.0,62,94,38597
4,2003.0,65,105,41423
5,2004.0,88,124,46779
6,2005.0,89,139,55967
7,2006.0,109,184,68302
8,2007.0,38,50,18258


In [14]:
less_2 = ''' SELECT aa.avg_rating as avg_rating_more_3, ff.avg_rating as avg_rating_rest from
            (SELECT avg(rating) as avg_rating
            FROM ratings
            where book_id in ( SELECT book_id
            FROM reviews
            group by book_id
            having count(distinct(review_id)) > 3)) as aa
            
            full outer join 
            
            (SELECT avg(rating) as avg_rating
            FROM ratings
            where book_id in ( SELECT book_id
            FROM reviews
            group by book_id
            having count(distinct(review_id)) < 3)) as ff on aa.avg_rating = ff.avg_rating
                        
        '''
pd.io.sql.read_sql(less_2, con = engine)

,avg_rating_more_3,avg_rating_rest
0,3.949505,NaN
1,NaN,3.906279


## Описание выводов

Благодаря исследованию базы данных, удалось выявить, что: после 1 января 2000 года было выпущено 819 книг. На каждую книгу в приложении есть обзоры, средние оценки которых 1,5-5,0. Самое большое количество книг выпущено издательством Penguin Books, это книги в которых более 50 страниц. Самую высокую оценку имеет автор Джоан Роулинг. Ее оценка 4,3. В среднем по 24 книги оценивают активные бользователи сервиса. 

Книги, которые оценивали более 3 человек лидируют по рейтингу.